# Batch Scoring Notebook
In this notebook, we step through batch scoring against a deployed machine learning model. There are multiple uses for batch scoring but our focus in this scenario is batch scoring so that Watson Openscale can collect enough data points to be able to provide adequate metrics on fairness and accuracy of the trained model.

In [18]:
import pandas as pd

In [2]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

## WML Deployed Model
In this section, we setup the configuration to access Watson Machine Learning deployed model.

In the next cell, please provide the Watson Machine Learning credentials which you can get by navigating to your IBM Cloud account, finding your Watson Machine Learning service, and clicking on "Service Credentials".
```
wml_credentials = {
    "apikey": "########",
    "instance_id": "#########",
    "url": "##########"
}
```

In [3]:
# Fill out apikey, instance_id, and url values from your WML service credentials
wml_credentials = {
    "apikey": "#########",
    "instance_id": "##########",
    "url": "#########"
}

In [4]:
# The code was removed by Watson Studio for sharing.

In [5]:
client = WatsonMachineLearningAPIClient( wml_credentials )

In [12]:
client.deployments.list()

------------------------------------  ---------------------------------------  ------  --------------  ------------------------  -----------------  -------------
GUID                                  NAME                                     TYPE    STATE           CREATED                   FRAMEWORK          ARTIFACT TYPE
a21433bb-fc7b-41bc-830e-b5096691ca0c  Customer Churn ICP4D labuser1            online  DEPLOY_SUCCESS  2019-10-09T18:10:21.313Z  n/a                function
16b69e68-fbfa-43ce-b471-86c4e984c29b  KoL-model-icp4d-depl-jk                  online  DEPLOY_SUCCESS  2019-05-17T20:32:30.254Z  mllib-2.3          model
4b3bbcbb-285c-48bd-8f1f-a3ab2ca8f051  RF Predict Churn SparkML Jan2019         online  DEPLOY_SUCCESS  2019-02-11T17:58:44.425Z  mllib-2.1          model
7faf7412-97c2-4cae-be62-e086c1d0d3fb  Predict Employee Attrition ALL Features  online  DEPLOY_SUCCESS  2018-12-07T19:29:38.895Z  scikit-learn-0.19  model
------------------------------------  -------------------

#### Identify the right deployment id from the above list corresponding to the deployment to which you want to send the scoring request. 
#### Use the same deployment id to get deployment details in the next cell

In [13]:
deployment_id = '###########'
deployment_details = client.deployments.get_details(deployment_id)

In [14]:
# Extract the scoring end point for the churn prediction model
churn_scoring_url = deployment_details['entity']['scoring_url']
print(churn_scoring_url)

https://us-south.ml.cloud.ibm.com/v3/wml_instances/e499ea96-f78a-42fe-a916-ff7dc1a2bca6/deployments/a21433bb-fc7b-41bc-830e-b5096691ca0c/online


## Scoring Data
In this section, we download the scoring data which we will send to the deployed customer churn prediction machine learning model.

The data is stored as a csv file on github so it can be loaded directly into a pandas dataframe using read_csv functionality.


In [30]:
csvurl = "https://raw.githubusercontent.com/joe4k/customerchurnopenscale/master/data/CustomerChurnScoringData.csv"
churn_data_daily = pd.read_csv(csvurl,header=0)


In [31]:
churn_data_daily.shape

(533, 16)

In [32]:
# drop ID column
churnDF = churn_data_daily.drop(['ID'],axis=1)

In [33]:
churnDF.head()

,GENDER,STATUS,CHILDREN,ESTINCOME,CAROWNER,AGE,LONGDISTANCE,INTERNATIONAL,LOCAL,DROPPED,PAYMETHOD,LOCALBILLTYPE,LONGDISTANCEBILLTYPE,USAGE,RATEPLAN
0,F,M,2,52004.8,N,25.140000,5,0,23,0,CH,Budget,Intnl_discount,28,1
1,M,M,1,75004.5,N,64.800000,26,0,32,0,CC,Budget,Intnl_discount,58,1
2,M,M,0,19749.3,N,60.366667,20,0,13,0,CC,Budget,Standard,34,3
3,M,S,1,57626.9,Y,43.906667,9,0,38,0,CC,Budget,Standard,48,2
4,F,S,0,78851.3,N,48.373333,0,0,28,0,CC,FreeLocal,Standard,29,4


In [34]:
deployment_details = client.deployments.get_details(deployment_id)
churn_scoring_url = deployment_details['entity']['scoring_url']


In [35]:
flds = churnDF.columns.tolist()
print(flds)

['GENDER', 'STATUS', 'CHILDREN', 'ESTINCOME', 'CAROWNER', 'AGE', 'LONGDISTANCE', 'INTERNATIONAL', 'LOCAL', 'DROPPED', 'PAYMETHOD', 'LOCALBILLTYPE', 'LONGDISTANCEBILLTYPE', 'USAGE', 'RATEPLAN']


In [36]:
values = churnDF.values.tolist()

In [37]:
print(len(values))

533


In [38]:
flds = churnDF.columns.tolist()
values = churnDF.values.tolist()
payload = {
    "fields": flds,
    "values": values
}

In [39]:
churnDF.shape

(533, 15)

In [40]:
result = client.deployments.score(churn_scoring_url, test_data)
print(result)

ApiRequestFailure: Failure during scoring. (POST https://us-south.ml.cloud.ibm.com/v3/wml_instances/e499ea96-f78a-42fe-a916-ff7dc1a2bca6/deployments/a21433bb-fc7b-41bc-830e-b5096691ca0c/online)
Status code: 504, body: upstream request timeout

## Conclusion
You have completed running batch scoring task for monitoring a churn prediction machine learning model.